# Inverse Problems Exercises: 2022s s03 (non-physics)
https://www.umm.uni-heidelberg.de/miism/

## Notes
* Please **DO NOT** change the name of the `.ipynb` file. 
* Please **DO NOT** import extra packages to solve the tasks. 
* Please put the `.ipynb` file directly into the `.zip` archive without any intermediate folder. 

## Please provide your personal information
* full name (Name): 

Maximilian Richter

## D01b: Wiener filter

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
file_gaussian = 'file_gaussian.npz'

with np.load(file_gaussian) as data:
    f_true = data['f_true']
    h_psf = data['h_psf']
    list_gn = data['list_gn']

### Imaging model
The imaging model can be represented by
$$
g = h \otimes f_\text{true}
= Af_\text{true}
= \mathcal{F}^{-1}\{ \mathcal{F}\{h\} \mathcal{F}\{f_\text{true}\} \},
$$
$$
g' = g + \epsilon.
$$
* $f_\text{true}$ is the input signal
* $h$ is the point spread function (kernel)
* $\otimes$ is the convolution operator
* $A$ is the Toeplitz matrix of $h$
* $\mathcal{F}$ and $\mathcal{F}^{-1}$ are the Fourier transform operator and inverse Fourier transform operator
* $\epsilon$ is the additive Gaussian noise
* $g$ is the filtered signal
* $g'$ is the noisy signal

### Fourier transform of the kernel
Implement the Fourier transform of the kernel $\mathcal{F}\{h\}$
* Given the kernel $h$
* Given the length of the transformed kernel $l$
* Pad zeros to both sides of the kernel
* Adjust the kernels as long as $l$
* Shift the origin of the kernels to the first element of the array
* Apply the Fourier transform to the shifted padded kernel (using `numpy.fft.fft()`)
* Implement the function `fft_kernel()` (using `numpy.array`)

Calculate the transformed kernel
* Apply the transform to `h_psf`
* Return the outputs of with the length of $100$, $1000$, $10000$, respectively
* Save the outputs in the variable `list_h_fft` (as `list` of `numpy.array`)

Display the result
* Plot the absolute value of the outputs in `list_h_fft` in the same order of the parameter options in the subplots of `axs`
* Plot the outputs properly in the frequency domain
* Plot the outputs with the marker "+"
* Add proper titles to the subplots of `axs`

In [ ]:
def fft_kernel(kernel, length):
    """Compute the discrete Fourier Transform of the kernel.

    :param kernel: 1d kernel of the system
    :param length: length of the transformed kernel
    :returns: Transformed kernel
    """
    padded = np.pad(kernel, int(length/2-kernel.shape[0]/2+1))
    shifted = np.roll(padded[1:], int(padded.shape[0]/2+1))
    return np.fft.fft(shifted)

lengths = [100, 1000, 10000]
list_h_fft = [fft_kernel(h_psf, n) for n in lengths]

fig, axs = plt.subplots(3, 1, figsize=(15, 10))
fig.suptitle('Fourier transform')

for i in range(len(list_h_fft)):
    axs[i].plot(np.abs(list_h_fft[i]), "+")
    axs[i].set_title("Length of {}".format(lengths[i]))

In [ ]:
# This cell contains hidden tests.


In [ ]:
# This cell contains hidden tests.


### Mean squared error
Implement the mean squared error (MSE)
$$
\operatorname{MSE}(\tilde f)=\frac{1}{n}\sum_{i=1}^n(f_{\text{true}i}-\tilde f_i)^2
$$
* Given the true signal $f_\text{true}$
* Given the estimate $\tilde f$
* Implement the function `mean_squared_error()` (using `numpy.array`)


In [ ]:
def mean_squared_error(f_true, f_est):
    """ Compute the mean squared error comparing to the true signal:

    :param f_true: True signal.
    :param f_est: Estimate of the signal.
    :returns: Mean squared error.
    """
    return np.mean((f_true - f_est)**2)

In [ ]:
# This cell contains hidden tests.


### Inverse filter
Implement the inverse filter
$$
\tilde{f} = \mathcal{F}^{-1}\{ \frac{1}{\mathcal{F}\{h\} + s^2} \cdot \mathcal{F}\{g'\} \}
$$
* Given the kernel $h$
* Given the noisy signal $g'$
* Given the small positive parameter $s^2$
* Transform the kernel by `fft_kernel()`
* Implement the function `inverse_filter()` (using `numpy.array`)

Apply the inverse filter
* Apply the inverse filter to the noisy signals in `list_gn`
* Return the outputs with $s^2$ of $0.5$, $0.1$, $0.02$, respectively 
* Save the outputs in the variable `list_f_inv` (as `list` of `numpy.array`)

Display the result
* Plot the outputs in `list_f_inv` in the same order of the parameter options in the subplots of `axs`
* Show the cases of the same noisy signal in the same subplot column
* Show the cases with the same $s^2$ in the same subplot row
* Plot the corresponding noisy signal in each subplot
* Plot the input signal `f_true` in each subplot
* Show the legend in each subplot
* Show the case information in the titles to the subplots
* Show the mean squared error of each output comparing to `f_true` in the titles to the subplots

In [ ]:
def inverse_filter(kernel, signal, s_sqr):
    """Apply an inverse filter kernel to a signal to deblur it.
    Use a small positive parameter s_sqr to avoid division by zero.

    :param kernel: 1d kernel of the system
    :param signal: 1d signal, which should be filtered
    :param s_sqr: Small positive parameter
    :returns: Filtered signal
    """
    return np.fft.ifft(np.fft.fft(signal) / (fft_kernel(kernel, signal.shape[0]) + s_sqr))

list_f_inv = []
s_sqr = [0.5, 0.1, 0.02]
for i in range(3):
    for j in range(3):
        list_f_inv.append(inverse_filter(h_psf, list_gn[j], s_sqr[i]))

fig, axs = plt.subplots(3, 3, figsize=(15, 15))
fig.suptitle('Inverse filter')

for i in range(3):
    for j in range(3):
        gn = list_gn[j]
        f_est = list_f_inv[i*3+j].real
        mse = mean_squared_error(f_true, f_est)
        axs[i,j].plot(f_true, label="$f_{true}$", color="black")
        axs[i,j].plot(gn, label="$g$", color="blue", alpha=0.8)
        axs[i,j].plot(f_est, label="Inverse", color="darkorange", alpha=0.8)
        axs[i,j].set_title("Signal {}, $s^2$ = {}, MSE = {}".format(j, s_sqr[i], np.round(mse)))
        axs[i,j].legend()

In [ ]:
# This cell contains hidden tests.


In [ ]:
# This cell contains hidden tests.


### Describe the influence of $s^2$ on the result

The results of this experiment suggests that the size of $s^2$ has a vast influence on the reconstruction of the signal by means of a inverse filter. This means in particular that both, very small and very big numbers for $s^2$ result in a similar high MSE. What differs is the source of these high errors. In the case of a big $s^2$ (0.5) the reconstructed signal is lower in amplitude than the original signal. This is due to the additional bias ontop of the theoretical inverse of the convolution. In the case of small $s^2$ (0.02), the division of small numbers result in a over amplified noise. Only for a "golden middle" $s^2$ of 0.1, the signal is not pushed down and the resulting noise is moderate. This means that $s^2$ has to be chosen by trading-off bias and variance.

### Wiener filter
Implement the Wiener filter
$$
\begin{align*}
\tilde{f} &= \mathcal{F}^{-1}\{ W \cdot \mathcal{F}\{g'\} \} \\
 &= \mathcal{F}^{-1}\{ \frac{\mathcal{F}\{h\}^*}{|\mathcal{F}\{h\}|^2 + \text{NSR}} \cdot \mathcal{F}\{g'\} \} \\
 &= \mathcal{F}^{-1}\{ \frac{1}{\mathcal{F}\{h\}} \cdot \frac{|\mathcal{F}\{h\}|^2}{|\mathcal{F}\{h\}|^2 + \text{NSR}} \cdot \mathcal{F}\{g'\} \} \\
\end{align*}
$$
* Given the kernel $h$
* Given the noisy signal $g'$
* Given the small positive parameter $\text{NSR}$
* Transform the kernel by `fft_kernel()`
* Implement the function `wiener_filter()` (using `numpy.array`)

Apply the Wiener filter
* Apply the Wiener filter to the noisy signals in `list_gn`
* Return the outputs with $\text{NSR}$ of $0.1$, $0.01$, $0.001$, respectively 
* Save the outputs in the variable `list_f_wiener` (as `list` of `numpy.array`)

Display the result
* Plot the outputs in `list_f_wiener` in the same order of the parameter options in the subplots of `axs`
* Show the cases of the same noisy signal in the same subplot column
* Show the cases with the same $\text{NSR}$ in the same subplot row
* Plot the corresponding noisy signal in each subplot
* Plot the input signal `f_true` in each subplot
* Show the legend in each subplot
* Show the case information in the titles to the subplots
* Show the mean squared error of each output comparing to `f_true` in the titles to the subplots

See
* https://en.wikipedia.org/wiki/Wiener_deconvolution

In [ ]:
def wiener_filter(kernel, signal, nsr):
    """Apply a wiener filer on the signal to deblur and denoise it.

    :param kernel: 1d kernel of the system
    :param signal: 1d blurred signal with noise
    :param nsr: Noise-to-signal ratio of the noise to the original signal
    :returns: Deblured and denoised signal
    """
    F_h = fft_kernel(kernel, signal.shape[0])
    abs_F_h_sqrd = np.abs(F_h)**2
    return np.fft.ifft(np.fft.fft(signal) * abs_F_h_sqrd / (F_h*(abs_F_h_sqrd + nsr)))


list_f_wiener = []
nsr = [0.1, 0.01, 0.001]
for i in range(3):
    for j in range(3):
        list_f_wiener.append(wiener_filter(h_psf, list_gn[j], nsr[i]))

fig, axs = plt.subplots(3, 3, figsize=(15, 15))
fig.suptitle('Wiener filter')

for i in range(3):
    for j in range(3):
        gn = list_gn[j]
        f_est = list_f_wiener[i*3+j].real
        mse = mean_squared_error(f_true, f_est)
        axs[i,j].plot(f_true, label="$f_{true}$", color="black")
        axs[i,j].plot(gn, label="$g$", color="blue", alpha=0.7)
        axs[i,j].plot(f_est, label="Wiener", color="darkorange")
        axs[i,j].set_title("Signal {}, NSR = {}, MSE = {}".format(j, nsr[i], np.round(mse)))
        axs[i,j].legend()

In [ ]:
# This cell contains hidden tests.


In [ ]:
# This cell contains hidden tests.


### Compare the results using the Wiener filter to the ones using the inverse filter

The Wiener filter performs significantly different in all cases than the plain inverse filter. This can be seen for example by comparing the MSE, which is lower (and therefore better) for all cases. The Wiener filter seems to change slower for different values for the NSR, i.e. the interval with the optimal NSR value is larger than in the case of the inverse filter. 
The Wiener filter is also more robust against noise in the filtered data. While the inverse filter struggles enormously, the Wiener filter is still able to make a decent reconstruction.
In summary, the wiener-filter inverts the convolution with the Gaussian PSF as good or even better than the inverse filter, and is furthermore less prone to bad choices of the additional bias and robust against noise. While the inverse filter is reconstructing the signal before the convolution very well for the right choice of $s^2$, it can result in useless noise if too small or too high. This makes the wiener filter better in practice, when this is a problem and a little more computational effort is acceptable.  

# Additional Comments

While the exercise was really nice and educational, it was rather confusing that the source of the signal was not known. I think it would be much more illustrative if the model for the data creation would be included in the code, so that we really understand the aim of inverse filtering. Reasons for which i think this would benefit the understanding is that i did not realize until the end of this exercise that we have to "reconstruct the noise" (so to say). I first thought (intuitively) that the less noisy reconstruction is the better one, while in deconvoluting a Gaussian kernel the aim is to regain the sharp edges. As an example i have programmed myself this little example in which the differences of the Inverse and Wiener Filter get quite clear (at least in my opinion). Here it seems also, that the inverse filter is doing a better job at deconvoluting the signal, in contrary to the examples above.  Anyways, have fun correcting the exercises :)

In [ ]:
lin = np.linspace(0,10,100)
data = np.sin(lin)+np.random.normal(0,0.5,lin.shape[0])

fig = plt.figure(figsize=(20,10))
plt.plot(data, label="True Data", color="black")
plt.plot(np.convolve(data, h_psf, mode="same"), label="Filtered Signal")
plt.plot(inverse_filter(h_psf, data, 0.5).real, label="Inverse Filter")
plt.plot(wiener_filter(h_psf, data, 0.01).real, label="Wiener Filter")
plt.legend()